In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import lightgbm as lgb
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [2]:
df_train = pd.read_csv("statistics_10_train.csv" , sep = ',')
df_test = pd.read_csv("statistics_10_test.csv" , sep = ',')

In [3]:
X_train = df_train[['Kurtosis', 'Impulse factor', 'RMS', 'Margin factor', 'Skewness',
               'Shape factor', 'Peak to peak', 'Crest factor']].values
y_train = df_train['Tipo'].values
X_test = df_test[['Kurtosis', 'Impulse factor', 'RMS', 'Margin factor', 'Skewness',
               'Shape factor', 'Peak to peak', 'Crest factor']].values
y_test = df_test['Tipo'].values

In [4]:
from hyperopt import fmin, atpe, tpe, STATUS_OK, STATUS_FAIL, Trials
from hyperopt import hp
from hyperopt import space_eval
class HPOpt(object):

    def __init__(self, x_train, x_test, y_train, y_test):
        self.x_train = x_train
        self.x_test  = x_test
        self.y_train = y_train
        self.y_test  = y_test

    def process(self, fn_name, space, trials, algo, max_evals):
        fn = getattr(self, fn_name)
        try:
            result = fmin(fn=fn, space=space, algo=algo, max_evals=max_evals, trials=trials)
        except Exception as e:
            return {'status': STATUS_FAIL,
                    'exception': str(e)}
        return result, trials

    def lgb_clas(self, para):
        clf = lgb.LGBMClassifier(**para['clas_params'])
        return self.train_clf(clf, para)
    
    def train_clf(self, clf, para):
        clf.fit(self.x_train, self.y_train,
                eval_set=[(self.x_train, self.y_train), (self.x_test, self.y_test)], 
                verbose = False, early_stopping_rounds = 20)
        pred = clf.predict(self.x_test)
        loss = para['loss_func'](self.y_test, pred)
        return {'loss': loss, 'status': STATUS_OK}

In [5]:
from sklearn.metrics import accuracy_score
lgb_clas_params = {
    'learning_rate':    hp.choice('learning_rate',    np.arange(0.001, 0.5, 0.001)),
    'max_depth':        hp.choice('max_depth',        np.arange(5, 10, 1, dtype=int)),
    'min_child_weight': hp.choice('min_child_weight', np.arange(0, 10, 1)),
    'min_data_in_leaf': hp.choice('min_data_in_leaf', np.arange(0, 10, 1)),
    'subsample':        hp.choice('subsample',        np.arange(0.1, 1, 0.05)),
    'n_estimators':     hp.choice('n_estimators',     np.arange(10, 200, 10, dtype=int)),
    'num_leaves':       hp.choice('num_leaves',       np.arange(5, 51, 1, dtype=int)),
    }

lgb_para = dict()
lgb_para['clas_params'] = lgb_clas_params
lgb_para['loss_func' ] = lambda y, pred: accuracy_score(y, pred)# squared = False)
lgb_para["max_evals"] = 100

In [6]:
# Optimización 
obj = HPOpt(X_train, X_test, y_train, y_test)

lgb_opt = obj.process(fn_name='lgb_clas', space=lgb_para, trials=Trials(), algo=tpe.suggest, max_evals=lgb_para["max_evals"])
parametros = space_eval(lgb_clas_params, lgb_opt[0])

100%|██████████████████████████████████████████████| 100/100 [01:59<00:00,  1.19s/trial, best loss: 0.8333333333333334]


In [7]:
clf = lgb.LGBMClassifier()
clf.set_params(**parametros) 
clf.fit(X_train, y_train)

LGBMClassifier(learning_rate=0.342, max_depth=9, min_child_weight=0,
               min_data_in_leaf=7, n_estimators=90, num_leaves=33,
               subsample=0.15000000000000002)

In [8]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
pred = clf.predict(X_test)
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

[[27  3  0]
 [ 0 30  0]
 [ 0  1 29]]
              precision    recall  f1-score   support

       Inner       1.00      0.90      0.95        30
       Outer       0.88      1.00      0.94        30
        Sano       1.00      0.97      0.98        30

    accuracy                           0.96        90
   macro avg       0.96      0.96      0.96        90
weighted avg       0.96      0.96      0.96        90



In [9]:
clf = lgb.LGBMClassifier(n_estimators = 100, learning_rate = 0.01, min_data_in_leaf = 0)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
target_names = ['Inner', 'Outer', 'Healthy']
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred, target_names = target_names))

[[29  1  0]
 [ 0 30  0]
 [ 0  3 27]]
              precision    recall  f1-score   support

       Inner       1.00      0.97      0.98        30
       Outer       0.88      1.00      0.94        30
     Healthy       1.00      0.90      0.95        30

    accuracy                           0.96        90
   macro avg       0.96      0.96      0.96        90
weighted avg       0.96      0.96      0.96        90



In [10]:
pred_train = clf.predict(X_train)
print(confusion_matrix(y_train, pred_train))
print(classification_report(y_train, pred_train, target_names = target_names))

[[90  0  0]
 [ 0 90  0]
 [ 0  0 90]]
              precision    recall  f1-score   support

       Inner       1.00      1.00      1.00        90
       Outer       1.00      1.00      1.00        90
     Healthy       1.00      1.00      1.00        90

    accuracy                           1.00       270
   macro avg       1.00      1.00      1.00       270
weighted avg       1.00      1.00      1.00       270

